In [2]:
%run "../00_project_config.ipynb"

import pathlib as pl
import pandas as pd

local_mount = pl.Path(
    "/mounts/hilbert/project"
)

# ordered by priority to ensure that path files
# for batch 1 are not updated to newer phasing
# pipeline release for batch 2
gaf_paths_folders = [
    local_mount.joinpath(
        "projects/medbioinf/projects/mihen108/test_wd/rukki_b62fe4"
    ),
    local_mount.joinpath(
        "projects/medbioinf/projects/mihen108/test_wd/rukki_v0.3.1-alpha"
    ),
]

sample_sheet_file = pl.Path(
    "../../samples/vrk_unps_samples.tsv"
)

sample_sheet = pd.read_csv(
    sample_sheet_file, sep="\t",
    header=0, comment="#"
)

ignore_samples = [
    "HG00096",  # DEBUG: 2023-09-13 - ONT or Verkko issue
    "HG00733red1",
    "HG00733red2",
    "HG00733R1",
    "HG00733R2",
    "HG03683stage2",
    "HG03683stage4",
    "NA19983stage4"
]

gaf_files = dict()
not_updated_paths = 0
for path_folder in gaf_paths_folders:
    for path_file in local_mount.joinpath(path_folder).glob("**/*_rukki_paths.gaf"):
        sample = path_file.name.split("_")[0]
        if sample in ignore_samples:
            print(f"Skipping {sample}")
            continue
        remote_file_path = str(path_file).replace(str(local_mount), "/gpfs/project")
        if sample not in gaf_files:
            gaf_files[sample] = remote_file_path
        else:
            not_updated_paths += 1
            print(f"Not updating phasing paths for sample {sample}")

assert not_updated_paths == 39, f"Batch 1 size of 39 / mismatch: {not_updated_paths}"

sample_sheet["target"] = "sseq"
sample_sheet["phasing_paths"] = "n/a"
sample_sheet["phasing_paths"] = sample_sheet["sample"].replace(gaf_files)

# remove samples w/o phasing paths
sample_sheet = sample_sheet.loc[sample_sheet["sample"] != sample_sheet["phasing_paths"], :].copy()

sample_sheet = sample_sheet[["sample", "target", "phasing_paths", "hifi", "ont"]]

sseq_sample_sheet_file = pl.Path(
    "../../samples/vrk_sseq_samples.tsv"
)

with open(sseq_sample_sheet_file, "w") as tsv:
    _ = tsv.write(f"# {TODAY}\n")
    _ = tsv.write(f"# {sample_sheet.shape[0]} samples\n")
    sample_sheet.to_csv(
        tsv, sep="\t", header=True, index=False
    )



Skipping HG00733red2
Skipping HG03683stage2
Skipping HG00733red1
Skipping HG03683stage4
Skipping NA19983stage4
Not updating phasing paths for sample HG03248
Skipping HG00733red2
Not updating phasing paths for sample HG02587
Not updating phasing paths for sample HG00732
Not updating phasing paths for sample HG03683
Not updating phasing paths for sample HG02818
Not updating phasing paths for sample HG04036
Skipping HG03683stage2
Not updating phasing paths for sample HG00733
Not updating phasing paths for sample NA19317
Not updating phasing paths for sample NA19983
Not updating phasing paths for sample NA19331
Not updating phasing paths for sample HG03452
Not updating phasing paths for sample NA19434
Not updating phasing paths for sample HG01114
Not updating phasing paths for sample HG02666
Not updating phasing paths for sample HG01352
Skipping HG00733red1
Not updating phasing paths for sample HG00358
Not updating phasing paths for sample HG02769
Skipping HG03683stage4
Not updating phasin